In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "showcase"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [4]:
function get_vs(folder)
    files = filter(x -> occursin(".txt", x), readdir(folder))
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[1]) for i in 1:n]
end

function get_as(folder)
    files = filter(x -> occursin(".txt", x), readdir(folder))
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[2]) for i in 1:n]
end

function get_cs(folder)
    files = filter(x -> occursin(".txt", x), readdir(folder))
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[3]) for i in 1:n]
end

function get_xs(folder)
    files = filter(x -> occursin(".txt", x), readdir(folder))
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[4]) for i in 1:n]
end

function get_ols(folder)
    files = filter(x -> occursin(".txt", x), readdir(folder))
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[5]) for i in 1:n]
end

function get_fsols(folder)
    files = filter(x -> occursin(".txt", x), readdir(folder))
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[6]) for i in 1:n]
end

function get_thetas(folder)
    files = filter(x -> occursin(".txt", x), readdir(folder))
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[7]) for i in 1:n]
end

function txt_to_points(input_file)
    [[parse(Float64, s) for s in split(p," ")] for p in readlines(input_file)[3:end]]
end

function get_configurations_from_folder(folder)
    files = filter(x -> occursin(".txt", x), readdir(folder))
    realizations = Vector{Matrix{Float32}}(undef, length(files))
    for file in files
        centers = txt_to_points("$(folder)$(file)")
        realizations[parse(Int, split(file, ".")[1])+1] = transpose(permutedims(hcat(centers...)))
    end
    realizations
end

function folder_of_txts_to_jld2_with_interfaces(folder)
    selectionname = split(folder, "/")[end-2]
    idx = split(folder, "/")[end-1]
    realizations = get_configurations_from_folder(folder)
    interface_with_persistence = [MorphoMol.Energies.get_interface_diagram_and_geometry([e for e in eachcol(r)], 1206) for r in realizations]
    barycentric_subdivisions = [barycenters for (barycenters,_,_) in interface_with_persistence]
    ifils = [ifil for (_,ifil,_) in interface_with_persistence]
    idgms = [dgms for (_,_,dgms) in interface_with_persistence]
    Es = get_fsols(folder)
    thetas = get_thetas(folder)
    Vs = get_vs(folder) 
    As = get_as(folder)
    Cs = get_cs(folder)
    Xs = get_xs(folder)
    OLs = get_ols(folder)
    mkpath("../../Data/paper2/measure_paths/jld2s/$(selectionname)_interface/")
    @save "../../Data/paper2/measure_paths/jld2s/$(selectionname)_interface/$(idx).jld2" realizations barycentric_subdivisions ifils idgms Es thetas Vs As Cs Xs OLs
end

function folder_of_txts_to_jld2_with_persistence(folder)
    selectionname = split(folder, "/")[end-2]
    idx = split(folder, "/")[end-1]
    realizations = get_configurations_from_folder(folder)
    pdgms = [MorphoMol.Energies.get_persistence_diagram([e for e in eachcol(r)]) for r in realizations]
    pdgms = [[dgm[1], dgm[2], dgm[3]] for dgm in pdgms]
    Es = get_fsols(folder)
    thetas = get_thetas(folder)
    Vs = get_vs(folder) 
    As = get_as(folder)
    Cs = get_cs(folder)
    Xs = get_xs(folder)
    OLs = get_ols(folder)
    mkpath("../../Data/paper2/measure_paths/jld2s/$(selectionname)_persistence/")
    @save "../../Data/paper2/measure_paths/jld2s/$(selectionname)_persistence/$(idx).jld2" realizations pdgms Es thetas Vs As Cs Xs OLs
end

#@assert false # THIS TAKES SO LONG! Make sure to only generate folders not already existing!!!
for (root, dirs, _) in walkdir("../../Data/paper2/measure_paths/txts/selection/")
    for sd in dirs 
        folder_of_txts_to_jld2_with_interfaces("$(root)$(sd)/")
    end
end